## CIFAR-100 이미지 추출
### 클라이언트별 IID

In [ ]:
import matplotlib.font_manager
system_fonts = matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
system_fonts

In [ ]:
import os

import tensorflow as tf
import numpy as np
import pandas as pd
import PIL.Image as Image

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar100.load_data()
images = np.concatenate((train_images, test_images))
labels = np.concatenate((train_labels, test_labels))

In [ ]:
labels[:10]

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/datasets/cifar100/load_data
# https://www.cs.toronto.edu/~kriz/cifar.html
# https://github.com/keras-team/keras/issues/2653
labelnames_fine = ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 
                   'bed', 'bee', 'beetle', 'bicycle', 'bottle', 
                   'bowl', 'boy', 'bridge', 'bus', 'butterfly', 
                   'camel', 'can', 'castle', 'caterpillar', 'cattle', 
                   'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 
                   'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 
                   'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 
                   'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 
                   'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 
                   'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 
                   'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 
                   'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 
                   'plain', 'plate', 'poppy', 'porcupine', 'possum', 
                   'rabbit', 'raccoon', 'ray', 'road', 'rocket', 
                   'rose', 'sea', 'seal', 'shark', 'shrew', 
                   'skunk', 'skyscraper', 'snail', 'snake', 'spider', 
                   'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 
                   'tank', 'telephone', 'television', 'tiger', 'tractor', 
                   'train', 'trout', 'tulip', 'turtle', 'wardrobe', 
                   'whale', 'willow_tree', 'wolf', 'woman', 'worm']
labelnames_coarse = ['aquatic_mammals', 'fish', 'flowers', 'food_containers', 
                     'fruit_and_vegetables', 'household_electrical_devices', 
                     'household_furniture', 'insects', 'large_carnivores', 
                     'large_man-made_outdoor_things', 'large_natural_outdoor_scenes', 
                     'large_omnivores_and_herbivores', 'medium_mammals', 'non-insect_invertebrates', 
                     'people', 'reptiles', 'small_mammals', 'trees', 'vehicles_1', 'vehicles_2']
labelnames_map = {'aquatic_mammals': ['beaver', 'dolphin', 'otter', 'seal', 'whale'],
                  'fish': ['aquarium_fish', 'flatfish', 'ray', 'shark', 'trout'],
                  'flowers': ['orchid', 'poppy', 'rose', 'sunflower', 'tulip'],
                  'food_containers': ['bottle', 'bowl', 'can', 'cup', 'plate'],
                  'fruit_and_vegetables': ['apple', 'mushroom', 'orange', 'pear', 'sweet_pepper'],
                  'household_electrical_devices': ['clock', 'computer_keyboard', 'lamp', 'telephone', 'television'],
                  'household_furniture': ['bed', 'chair', 'couch', 'table', 'wardrobe'],
                  'insects': ['bee', 'beetle', 'butterfly', 'caterpillar', 'cockroach'],
                  'large_carnivores': ['bear', 'leopard', 'lion', 'tiger', 'wolf'],
                  'large_man-made_outdoor_things': ['bridge', 'castle', 'house', 'road', 'skyscraper'],
                  'large_natural_outdoor_scenes': ['cloud', 'forest', 'mountain', 'plain', 'sea'],
                  'large_omnivores_and_herbivores': ['camel', 'cattle', 'chimpanzee', 'elephant', 'kangaroo'],
                  'medium_mammals': ['fox', 'porcupine', 'possum', 'raccoon', 'skunk'],
                  'non-insect_invertebrates': ['crab', 'lobster', 'snail', 'spider', 'worm'],
                  'people': ['baby', 'boy', 'girl', 'man', 'woman'],
                  'reptiles': ['crocodile', 'dinosaur', 'lizard', 'snake', 'turtle'],
                  'small_mammals': ['hamster', 'mouse', 'rabbit', 'shrew', 'squirrel'],
                  'trees': ['maple_tree', 'oak_tree', 'palm_tree', 'pine_tree', 'willow_tree'],
                  'vehicles_1': ['bicycle', 'bus', 'motorcycle', 'pickup_truck', 'train'],
                  'vehicles_2': ['lawn_mower', 'rocket', 'streetcar', 'tank', 'tractor'],
                 }
fine2coarse = { 0:  4,  1:  1,  2: 14,  3:  8,  4:  0,
                5:  6,  6:  7,  7:  7,  8: 18,  9:  3,
               10:  3, 11: 14, 12:  9, 13: 18, 14:  7,
               15: 11, 16:  3, 17:  9, 18:  7, 19: 11,
               20:  6, 21: 11, 22:  5, 23: 10, 24:  7,
               25:  6, 26: 13, 27: 15, 28:  3, 29: 15,
               30:  0, 31: 11, 32:  1, 33: 10, 34: 12,
               35: 14, 36: 16, 37:  9, 38: 11, 39:  5,
               40:  5, 41: 19, 42:  8, 43:  8, 44: 15,
               45: 13, 46: 14, 47: 17, 48: 18, 49: 10,
               50: 16, 51:  4, 52: 17, 53:  4, 54:  2,
               55:  0, 56: 17, 57:  4, 58: 18, 59: 17,
               60: 10, 61:  3, 62:  2, 63: 12, 64: 12,
               65: 16, 66: 12, 67:  1, 68:  9, 69: 19,
               70:  2, 71: 10, 72:  0, 73:  1, 74: 16,
               75: 12, 76:  9, 77: 13, 78: 15, 79: 13,
               80: 16, 81: 19, 82:  2, 83:  4, 84:  6,
               85: 19, 86:  5, 87:  5, 88:  8, 89: 19,
               90: 18, 91:  1, 92:  2, 93: 15, 94:  6,
               95:  0, 96: 17, 97:  8, 98: 14, 99: 13}
print(f'Fine labels: {len(labelnames_fine)}, Coarse labels: {len(labelnames_coarse)}')

In [ ]:
clients = 5
div = clients + 1

In [ ]:
noniid = []
for key, value in labelnames_map.items():
    noniid.append(value[0])
print(f'Test dataset has only: {noniid}')

In [ ]:
counter_fine = {}
counter_coarse = {}
output = os.path.abspath(os.path.expanduser('dataset'))
for idx, data in enumerate(zip(images, labels), start=0):
    image = Image.fromarray(data[0])
    fine = labelnames_fine[data[1][0]]
    coarse = labelnames_coarse[fine2coarse[data[1][0]]]
    num_fine = counter_fine.get(fine, 0)
    num_coarse = counter_coarse.get(coarse, 0)
    if fine in noniid and num_fine%2 == 0:
        party = 0
    else:
        party = (num_coarse%clients) + 1
    odir = os.path.join(output, f'{party}', coarse)
    os.makedirs(odir, exist_ok=True)
    opath = os.path.join(odir, f'{num_coarse:04d}.jpg')
    image.save(opath)
    counter_fine[fine] = num_fine + 1
    counter_coarse[coarse] = num_coarse + 1

## 클라이언트별 학습

In [ ]:
dataset_root = os.path.abspath(os.path.expanduser('dataset'))
with os.scandir(dataset_root) as it:
    for entry in it:
        if not entry.name.startswith('.') and entry.is_dir():
            print(entry, entry.path, entry.name)

In [ ]:
# Datasets
datasets = {}
dataset_root = os.path.abspath(os.path.expanduser('dataset'))
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
with os.scandir(dataset_root) as it:
    for entry in it:
        if not entry.name.startswith('.') and entry.is_dir():
            dataset = image_generator.flow_from_directory(entry.path,
                                                          classes=labelnames_coarse, 
                                                          target_size=(32, 32), 
                                                          shuffle=True)
            datasets[entry.name] = dataset
datasets.keys()

In [ ]:
model = tf.keras.models.Sequential([
                       tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
                       tf.keras.layers.MaxPooling2D((2, 2)),
                       tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                       tf.keras.layers.MaxPooling2D((2, 2)),
                       tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                       tf.keras.layers.Flatten(),
                       tf.keras.layers.Dense(64, activation='relu'),
                       tf.keras.layers.Dense(len(labelnames_coarse))])
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

In [ ]:
# Base model
odir = os.path.join('models', '0', '0')
model.save(odir)

In [ ]:
rounds = 10
epochs = 10

In [ ]:
for r in range(rounds+1):
    weights = []
    rpath = os.path.join('models', f'{r}')
    for c in range(1, clients+1):
        mpath = os.path.join(rpath, '0')
        model = tf.keras.models.load_model(mpath)
        history = model.fit(datasets[f'{c}'], epochs=epochs, verbose=0)
        opath = os.path.join(rpath, f'{c}')
        model.save(opath)
        if c == 1:
            weights = model.get_weights()
        else:
            for idx, weight in enumerate(model.get_weights()):
                weights[idx] = weights[idx] + weight
        print(f'Local train: round #{r} with clinent #{c}')
    for idx, weight in enumerate(weights):
        weights[idx] = weights[idx] / clients
    mpath = os.path.join(rpath, '0')
    model = tf.keras.models.load_model(mpath)
    model.set_weights(weights)
    ndir = os.path.join('models', f'{r+1}')
    npath = os.path.join(ndir, '0')
    model.save(npath)
    metric = model.evaluate(datasets['0'], verbose=0)
    print(f'Global aggregation: round #{r+1} for {metric}')